In [1]:
import pandas as pd
from sqlalchemy import create_engine, text
import os

In [2]:
# Oracle database connection details
or_user = os.environ['oracle_username']
or_pwd = os.environ['oracle_password']
or_host = os.environ['oracle_host']
or_port = os.environ['oracle_port']
or_service = os.environ['oracle_service']


In [3]:
# PostgreSQL database connection details
pg_user = os.environ['postgres_username']
pg_pwd = os.environ['postgres_password']
pg_host = os.environ['postgres_host']
pg_port = os.environ['postgres_port']
pg_database = os.environ['postgres_database']


In [4]:
# Create database connections
oracle_engine = create_engine(f'oracle+cx_oracle://{or_user}:{or_pwd}@{or_host}:{or_port}/{or_service}')
postgres_engine = create_engine(f'postgresql://{pg_user}:{pg_pwd}@{pg_host}:{pg_port}/{pg_database}')


In [5]:
# Get the maximum Last_updated timestamp from Postgres emp_table1
max_last_updated_query_pg = 'SELECT MAX("Last_updated") FROM emp_table9'
max_last_updated_pg = pd.read_sql(max_last_updated_query_pg, postgres_engine)
max_last_updated_pg_value = max_last_updated_pg.iloc[0, 0]  # Extracting the timestamp value
print("Max Postgres database last date value =", max_last_updated_pg_value)


Max Postgres database last date value = 2024-02-19 19:24:08


In [6]:
# Query differing data from Oracle emp_table1
query_diff_ora = f"""
    SELECT *
    FROM emp_table9
    WHERE "Last_updated" > TO_TIMESTAMP('{max_last_updated_pg_value}', 'YYYY-MM-DD HH24:MI:SS.FF')
"""
diff_ora = pd.read_sql(query_diff_ora, oracle_engine)
print(diff_ora)


   Emp_id        Emp_name                  Emp_dept    Emp_joining_date  \
0     990      Fergus Pym  Research and Development 2024-01-20 01:08:15   
1     998  Adda Greenhill                     Legal 2023-10-24 09:47:41   
2     999    Giraud Levee                  Training 2024-02-17 19:02:50   
3    1000     Garey Birks                  Training 2023-03-08 15:45:41   
4    1001      Kiran Bhai                  Training 2023-04-03 15:22:58   

             Last_updated  
0 2024-03-19 11:35:47.543  
1 2024-03-18 14:50:13.830  
2 2024-03-18 14:45:50.808  
3 2024-03-18 14:50:42.886  
4 2024-03-18 14:50:46.597  


In [7]:
# Check for existing records in PostgreSQL with primary key values matching those of the differing data
if not diff_ora.empty:
    primary_key_values = tuple(diff_ora['Emp_id'])
    existing_records_query = text('SELECT * FROM emp_table9 WHERE "Emp_id" IN :ids')
    existing_records_query = existing_records_query.bindparams(ids=primary_key_values)
    existing_records = pd.read_sql(existing_records_query, postgres_engine)
    print(primary_key_values)

    # Delete existing records from Postgres table if found
    if not existing_records.empty:
        with postgres_engine.connect() as connection:
            for index, row in existing_records.iterrows():
                delete_query = text(f'DELETE FROM emp_table9 WHERE "Emp_id" IN ({row["Emp_id"]})')
                connection.execute(delete_query)
                print(delete_query)
            print("Existing records deleted from emp_table9 in PostgreSQL.")

    # Load the differing data into PostgreSQL table emp_table9
    diff_ora.to_sql('emp_table9', postgres_engine, if_exists='append', index=False)
    print("Differing data loaded into emp_table9 in PostgreSQL database.")
else:
    print("No differing data found in Oracle. Nothing to load.")

(990, 998, 999, 1000, 1001)
DELETE FROM emp_table9 WHERE "Emp_id" IN (990)
DELETE FROM emp_table9 WHERE "Emp_id" IN (998)
DELETE FROM emp_table9 WHERE "Emp_id" IN (999)
DELETE FROM emp_table9 WHERE "Emp_id" IN (1000)
Existing records deleted from emp_table9 in PostgreSQL.
Differing data loaded into emp_table9 in PostgreSQL database.
